# InSAR Processing Using ASF HYP3 on-demand products

## Getting Started

The Alaska Satellite Facility (ASF) archives SAR and InSAR data on behalf of NASA. 
By searching on their [Vertex website](https://search.asf.alaska.edu/#/?maxResults=250), users can access and download the original data, or in some cases can run basic processing. 
Vertex also allows users to do basic SBAS processing, which produces interferograms that can be downloaded and processed to time-series. 
This is what we will in this exercise. 


The ASF User guide can be found [here](https://docs.asf.alaska.edu/vertex/manual/)

See the HYP3 SDK [here](https://hyp3-docs.asf.alaska.edu/using/sdk/)

In [2]:
!pip install hyp3_sdk


Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for hyp3_sdk from https://files.pythonhosted.org/packages/a0/05/63a2e2cfda7c7db84f04bc0a3c66107eb17e0aca2ea4e9faf36c1d9b6fc8/hyp3_sdk-7.7.1-py3-none-any.whl.metadata


In [1]:
import glob
import os
from pathlib import Path

import numpy as np
import rasterio
from hyp3_sdk import HyP3
from process_data import *

In [3]:
hyp3 = HyP3(username='maurer', password='$WqFxx47$!^#DRJT')
batch = hyp3.find_jobs()

Once you have complete jobs you can download the products to your machine

In [4]:
batch.complete()

True

### WARNING!!!
This step will download files from ASF. 
These files vary in size but are roughly ~0.5 GB each, so 200 will be ~100 GB. 
Make sure you have adequete space for downloading before starting! 
The good news is that file downloads don't repeat, so if the code stops for any reason you can restart from where it ended. 

In [5]:
# specify your download folder
dload_path_name = './asf_insar_data'

In [6]:
batch.download_files(dload_path_name, create=True)

  0%|          | 0/5623 [00:00<?, ?it/s]

S1AA_20240807T235630_20240819T235630_VVP012_INT80_G_ueF_2D47.zip:   0%|          | 0/292472388 [00:00<?, ?it/s…

S1AA_20240819T235630_20240924T235631_VVP036_INT80_G_ueF_AD26.zip:   0%|          | 0/294250034 [00:00<?, ?it/s…

S1AA_20240912T235630_20241006T235632_VVP024_INT80_G_ueF_FE1C.zip:   0%|          | 0/293793543 [00:00<?, ?it/s…

S1AA_20240702T235630_20240726T235629_VVP024_INT80_G_ueF_323D.zip:   0%|          | 0/292903903 [00:00<?, ?it/s…

...

S1AA_20221029T235630_20221122T235630_VVP024_INT80_G_ueF_11D6.zip:   0%|          | 0/289568385 [00:00<?, ?it/s…

S1AA_20230202T235626_20230310T235626_VVP036_INT80_G_ueF_57DF.zip:   0%|          | 0/291587553 [00:00<?, ?it/s…

S1AA_20220725T235627_20220818T235628_VVP024_INT80_G_ueF_311E.zip:   0%|          | 0/293660095 [00:00<?, ?it/s…

S1AA_20210928T235624_20211010T235624_VVP012_INT80_G_ueF_AFD9.zip:   0%|          | 0/290599768 [00:00<?, ?it/s…

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



### Steps
The first steps are to download and unzip the products in our working directory. In this case, I have already done this step. 

In [7]:
# change to your path name
%cd asf_insar_data

/mnt/stor/geob/jlmd9g/Projects/SLU/t92f120/asf_insar_data


In [8]:
!ls

S1AA_20200101T235556_20200113T235556_VVP012_INT80_G_ueF_48B4.zip
S1AA_20200101T235556_20200113T235556_VVP012_INT80_G_ueF_8B98.zip
S1AA_20200101T235556_20200125T235555_VVP024_INT80_G_ueF_CDDA.zip
S1AA_20200101T235556_20200125T235555_VVP024_INT80_G_ueF_E47F.zip
S1AA_20200101T235556_20200206T235555_VVP036_INT80_G_ueF_E19F.zip
S1AA_20200101T235556_20200206T235555_VVP036_INT80_G_ueF_E943.zip
S1AA_20200101T235621_20200113T235620_VVP012_INT80_G_ueF_1989.zip
S1AA_20200101T235621_20200113T235620_VVP012_INT80_G_ueF_2BF3.zip
S1AA_20200101T235621_20200125T235620_VVP024_INT80_G_ueF_5CEF.zip
S1AA_20200101T235621_20200125T235620_VVP024_INT80_G_ueF_F4E1.zip
S1AA_20200101T235621_20200206T235620_VVP036_INT80_G_ueF_4219.zip
S1AA_20200101T235621_20200206T235620_VVP036_INT80_G_ueF_8E59.zip
S1AA_20200113T235556_20200125T235555_VVP012_INT80_G_ueF_5889.zip
S1AA_20200113T235556_20200125T235555_VVP012_INT80_G_ueF_B72F.zip
S1AA_20200113T235556_20200206T235555_VVP024_INT80_G_ueF_346D.zip
S1AA_20200113T235556_2020

In [ ]:
# Unzip your folders
# Depending on your platform this may not work
!unzip '*.zip'

Archive:  S1AA_20200125T235555_20200301T235555_VVP036_INT80_G_ueF_5033.zip
   creating: S1AA_20200125T235555_20200301T235555_VVP036_INT80_G_ueF_5033/
  inflating: S1AA_20200125T235555_20200301T235555_VVP036_INT80_G_ueF_5033/S1AA_20200125T235555_20200301T235555_VVP036_INT80_G_ueF_5033_unw_phase.png.aux.xml  
  inflating: S1AA_20200125T235555_20200301T235555_VVP036_INT80_G_ueF_5033/S1AA_20200125T235555_20200301T235555_VVP036_INT80_G_ueF_5033_color_phase.kmz  
  inflating: S1AA_20200125T235555_20200301T235555_VVP036_INT80_G_ueF_5033/S1AA_20200125T235555_20200301T235555_VVP036_INT80_G_ueF_5033_amp.tif  
  inflating: S1AA_20200125T235555_20200301T235555_VVP036_INT80_G_ueF_5033/S1AA_20200125T235555_20200301T235555_VVP036_INT80_G_ueF_5033_lv_phi.tif.xml  
  inflating: S1AA_20200125T235555_20200301T235555_VVP036_INT80_G_ueF_5033/S1AA_20200125T235555_20200301T235555_VVP036_INT80_G_ueF_5033_water_mask.tif.xml  
  inflating: S1AA_20200125T235555_20200301T235555_VVP036_INT80_G_ueF_5033/S1AA_202001

In [14]:
!ls

S1AA_20200101T235556_20200113T235556_VVP012_INT80_G_ueF_48B4
S1AA_20200101T235556_20200113T235556_VVP012_INT80_G_ueF_48B4.zip
S1AA_20200101T235556_20200113T235556_VVP012_INT80_G_ueF_8B98
S1AA_20200101T235556_20200113T235556_VVP012_INT80_G_ueF_8B98.zip
S1AA_20200101T235556_20200125T235555_VVP024_INT80_G_ueF_CDDA
S1AA_20200101T235556_20200125T235555_VVP024_INT80_G_ueF_CDDA.zip
S1AA_20200101T235556_20200125T235555_VVP024_INT80_G_ueF_E47F
S1AA_20200101T235556_20200125T235555_VVP024_INT80_G_ueF_E47F.zip
S1AA_20200101T235556_20200206T235555_VVP036_INT80_G_ueF_E19F
S1AA_20200101T235556_20200206T235555_VVP036_INT80_G_ueF_E19F.zip
S1AA_20200101T235556_20200206T235555_VVP036_INT80_G_ueF_E943
S1AA_20200101T235556_20200206T235555_VVP036_INT80_G_ueF_E943.zip
S1AA_20200101T235621_20200113T235620_VVP012_INT80_G_ueF_1989
S1AA_20200101T235621_20200113T235620_VVP012_INT80_G_ueF_1989.zip
S1AA_20200101T235621_20200113T235620_VVP012_INT80_G_ueF_2BF3
S1AA_20200101T235621_20200113T235620_VVP012_INT80_G_ueF_2

As you can see, we have a number of folders that each contain a set of files, including unwrapped phase, unwrapped displacements, DEM, look vectors, etc. 

Each folder is named by the two dates that were used to generate that interferogram, so for example `S1AA_20220420T235621_20220514T235622_VVP024_INT80_G_ueF_CF44` is an interferogram for dates 4/20/2022 and 5/14/2024. 

In [15]:
# Count the number of interferograms we have downloaded
# If this line does not work on your machine feel free to skip
!ls -ld */ | wc -l  # this will count folders only

772


In [ ]:
!mkdir zipped

In [18]:
!mv *.zip zipped/.

mv: cannot stat '*.zip': No such file or directory


In [19]:
!ls

S1AA_20200101T235556_20200113T235556_VVP012_INT80_G_ueF_48B4
S1AA_20200101T235556_20200113T235556_VVP012_INT80_G_ueF_8B98
S1AA_20200101T235556_20200125T235555_VVP024_INT80_G_ueF_CDDA
S1AA_20200101T235556_20200125T235555_VVP024_INT80_G_ueF_E47F
S1AA_20200101T235556_20200206T235555_VVP036_INT80_G_ueF_E19F
S1AA_20200101T235556_20200206T235555_VVP036_INT80_G_ueF_E943
S1AA_20200101T235621_20200113T235620_VVP012_INT80_G_ueF_1989
S1AA_20200101T235621_20200113T235620_VVP012_INT80_G_ueF_2BF3
S1AA_20200101T235621_20200125T235620_VVP024_INT80_G_ueF_5CEF
S1AA_20200101T235621_20200125T235620_VVP024_INT80_G_ueF_F4E1
S1AA_20200101T235621_20200206T235620_VVP036_INT80_G_ueF_4219
S1AA_20200101T235621_20200206T235620_VVP036_INT80_G_ueF_8E59
S1AA_20200113T235556_20200125T235555_VVP012_INT80_G_ueF_5889
S1AA_20200113T235556_20200125T235555_VVP012_INT80_G_ueF_B72F
S1AA_20200113T235556_20200206T235555_VVP024_INT80_G_ueF_346D
S1AA_20200113T235556_20200206T235555_VVP024_INT80_G_ueF_C673
S1AA_20200113T235556_202

In [20]:
# some QA/QC
!ls S1*/*_unw_phase.tif | wc -l

771


In [21]:
# this should match the previous
!ls S1*/*_corr.tif | wc -l 

771


In [2]:
# check file sizes
files = glob.glob('asf_insar_data' + os.sep + 'S1' + '*' + os.sep + '*' + '_unw_phase.tif')
cfiles = glob.glob('asf_insar_data' + os.sep + 'S1' + '*' + os.sep + '*' + 'corr.tif')
print(len(files))
print(len(cfiles))

all_sizes, all_heights, all_widths, all_sizes = [], [], [], []
for f in files:
    with rasterio.open(f, 'r') as F:
        all_sizes.append((F.height, F.width))
        all_heights.append(F.height)
        all_widths.append(F.width)
        
print(np.mean(all_heights))
print(np.mean(all_widths))

771
771
2777.3813229571983
3538.284046692607


In [3]:
assert len(set(files))==len(files)

The next step is to make sure that all the interferograms have the same shape and geographic extent. 
We will ensure this using a simple code that uses rasterio to re-project all the interferograms to match a reference. 
For this code, you will need `rasterio` and `numpy` installed. 

In [8]:
# for k, f in enumerate(files):
#     with rasterio.open(f) as F:
#         plt.plot([k,k], F.

In [4]:
# The function to resize the images is called run_resampling
shp_file = '/home/jlmd9g/CEF/Projects/SLU/t92f120/asf_insar_data/stl_polygon/stl_polygon.shp'
run_resampling(path_to_shapefile = shp_file, data_dir='.') # this runs the program on the current directory

ERROR 1: PROJ: proj_create_from_database: Open of /home/jlmd9g/.conda/envs/mintpy/share/proj failed
  0%|          | 1/771 [00:04<56:00,  4.36s/it]

Currently running file #0 of 771.


  1%|▏         | 11/771 [00:45<49:29,  3.91s/it]

Currently running file #10 of 771.


  3%|▎         | 21/771 [01:25<49:25,  3.95s/it]

...

Currently running file #750 of 771.


 99%|█████████▊| 761/771 [1:49:40<01:27,  8.74s/it]

Currently running file #760 of 771.


100%|██████████| 771/771 [1:51:18<00:00,  8.66s/it]

Currently running file #770 of 771.


Once the resampling has finished, we can now proceed with time-series analysis as normal!
You can refer to the [InSAR lab](https://github.com/jlmaurer/GE6146/blob/master/notebooks/Lab4_InSAR.ipynb) for more details and plotting examples. 

In [7]:
# We'll use the same functions as for the lab
from makeTS import *

In [11]:
glob_path = Path(os.getcwd())
ifgList = list(glob_path.glob('**/*_unw_phase.tif'))
cohList = list(glob_path.glob('**/*_corr.tif'))

In [12]:
print(f'I found {len(ifgList)} interferograms')

I found 771 interferograms


In [ ]:
# Pull all the data
datePairs, dates = getDates(ifgList)
fracDates = np.array([dt2fracYear(d) for d in dates])
G = makeG(dates, datePairs)
xSize, ySize, dType, geoProj, trans, noDataVal, Nbands = readRaster(ifgList[0])
data = getData(ifgList,1)

In [6]:
print(f'Interferograms are {ySize}x{xSize} pixels in size')

Interferograms are 2841x3557 pixels in size


In [ ]:
# You'll need to make a plot of one of the dates or the average coherence so you can pick a reference point

In [7]:
# de-referencing
data = dereference(data, taxis=0,refCenter=[2500,2000],refSize=10)

In [8]:
# create the time-series data cube
tsArray = makeTS(G, data, fracDates)

In [9]:
# Convert to displacement in meters
tsArray = convertRad2meters(tsArray, lam=0.056) # wavelength lam is 5.6 cm for Sentinel-1

In [10]:
# Compute the mean velocity by fitting a line
vel = findMeanVel(tsArray, fracDates, 0)

In [11]:
vel.shape

(2841, 3557)

### Saving Data
Want to change this to use NETCDF and xarray, include time-series and coherence time-series in the files, maybe IFGs and coherence ifgs as well. 

In [1]:
import h5py


with h5py.File('velocity.h5') as f:
    for key in f.keys():
        print(key)

intercept
interceptStd
residue
velocity
velocityStd


## Tropospheric correction using RAiDER

TBD